In [2]:
%pip install pandas

  Using cached pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
Using cached numpy-1.24.4-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd

# Load the CSV
df = pd.read_csv("dataset/Flood_events.csv")

# Step 1: Filter for Quang Ngai
df_qn = df[df["Location"].str.contains("Quang Ngai", case=False, na=False)].copy()

# Focus on rows with missing start/end day but known month/year
df_qn["Start Day"] = df_qn["Start Day"].fillna(13)
df_qn["End Day"] = df_qn["End Day"].fillna(17)

# Step 2: Build datetime columns
df_qn["Start Date Full"] = (
    df_qn["Start Year"].astype(int).astype(str) + "-" +
    df_qn["Start Month"].astype(int).astype(str).str.zfill(2) + "-" +
    df_qn["Start Day"].astype(int).astype(str).str.zfill(2)
)

df_qn["End Date Full"] = (
    df_qn["End Year"].astype(int).astype(str) + "-" +
    df_qn["End Month"].astype(int).astype(str).str.zfill(2) + "-" +
    df_qn["End Day"].astype(int).astype(str).str.zfill(2)
)

# Step 3: Remove rows with invalid dates (NaNs from missing fields)
df_qn.dropna(subset=["Start Date Full", "End Date Full"], inplace=True)

# Convert to datetime for comparison and expansion
df_qn["Start Date Full"] = pd.to_datetime(df_qn["Start Date Full"], errors="coerce")
df_qn["End Date Full"] = pd.to_datetime(df_qn["End Date Full"], errors="coerce")

# Step 4: Detect inconsistent date rows
inconsistent_rows = df_qn[df_qn["Start Date Full"] > df_qn["End Date Full"]]

# Step 5: Generate all flood days from consistent data only
valid_rows = df_qn[df_qn["Start Date Full"] <= df_qn["End Date Full"]]

flood_days = []
for _, row in valid_rows.iterrows():
    flood_days.extend(pd.date_range(start=row["Start Date Full"], end=row["End Date Full"]))

# Optional: Print inconsistent rows for manual checking
print("🚨 Inconsistent date rows where Start > End:")
print(inconsistent_rows[[
    "DisNo.", "Start Year", "Start Month", "Start Day", 
    "End Year", "End Month", "End Day",
    "Start Date Full", "End Date Full", "Location"
]])

# Step 6: Finalize flood dates
flood_days = sorted(set(flood_days))
flood_daily_df = pd.DataFrame({"Flood Date": flood_days})

# Save to CSV
flood_daily_df.to_csv("flood_ground_truth.csv", index=False)

# Preview
print(flood_daily_df.head())


🚨 Inconsistent date rows where Start > End:
Empty DataFrame
Columns: [DisNo., Start Year, Start Month, Start Day, End Year, End Month, End Day, Start Date Full, End Date Full, Location]
Index: []
  Flood Date
0 1980-11-13
1 1980-11-14
2 1980-11-15
3 1980-11-16
4 1980-11-17
